# Whole genome sequencing analysis workflow

### Problem

Adaptive laboratory evolution was conducted on a S. cerevisiae co-culture of two cross-feeding strains (sHP063, sHP067), generating the evolved cross-feeding strains B2 (evolved sHP063) and R8 (evolved sHP067).
We want to understand what adaptation has the evolution process resulted in at a genomic level using the WGS data by comparing the evolved R8 and B2 strains against their non-evolved parentals (sHP067 and sHP063, respectively).
 
Samples are labelled as follows:
- BY4741, base strain (sHP063 and sHP067 were built from this “reference genome”)
- sHP063 (non-evolved)
- sHP067 (non-evolved)
- B2 (sHP063 evolved)
- R8 (sHP067 evolved)

![co-culture](co-culture.jpg)

Sequencing was performed on the Illumina platform, generating raw reads (or raw data) stored in FASTQ (.fq) format files. These files contain the sequencing reads along with base quality scores. Raw reads were filtered to produce clean reads, with the following criteria applied:

1. Paired reads are removed if either read contains adapter contamination.
2. Paired reads are removed if uncertain nucleotides (represented by "N") make up more than 10% of either read.
3. Paired reads are removed if low-quality nucleotides (base quality ≤ 5) constitute more than 50% of either read.

## Step 1: De novo genome assembly of ancestral strains

To identify genetic variations in the evolved strains relative to their ancestral, non-evolved parent strains, the first step is to reconstruct the reference FASTA sequence for each ancestral strain. This is achieved through de novo genome assembly, which builds a complete genome sequence from raw sequencing reads.

In [ ]:
# create and activate environment
conda install -c conda-forge mamba --yes

mamba create -n de_novo_assembly -c bioconda -c conda-forge fastqc spades megahit quast busco bowtie2 ragtag --yes
mamba create -n ragout_env -c bioconda -c conda-forge ragout --yes   # uses incompatible package specs with programs above
conda activate de_novo_assembly

In [ ]:
# assess sequence data quality
cd X204SC24082098-Z01-F002_01/01.RawData/sHP063/
fastqc sHP063_1.fq.gz sHP063_2.fq.gz -t 4

cd X204SC24082098-Z01-F002_01/01.RawData/sHP067/
fastqc sHP067_1.fq.gz sHP067_2.fq.gz -t 4

Overall the reads quality is good (as they had already been filtered by Novogene) so no need for trimming.

The next step is assembling the reads. I tested different assembly programs ([SPAdes](https://ablab.github.io/spades/) and [MEGAHIT](https://github.com/voutcn/megahit/wiki)) with different parameters and then compared the results. 

First I run an error-correction step with SPAdes, which tends to improve assembly results according to [this tutorial](https://astrobiomike.github.io/genomics/de_novo_assembly) I am following. This is the most computationally intensive step of the workflow (took up to 4h on the HPC as a submitted job).

In [ ]:
cd X204SC24082098-Z01-F002_01/01.RawData/sHP063/
spades.py -1 sHP063_1.fq.gz -2 sHP063_2.fq.gz -o spades_error_corrected_reads -t 50 -m 500 --only-error-correction

cd X204SC24082098-Z01-F002_01/01.RawData/sHP067/
spades.py -1 sHP067_1.fq.gz -2 sHP067_2.fq.gz -o spades_error_corrected_reads -t 50 -m 500 --only-error-correction

# -t : number of threads
# -m : RAM limit in Gb

Or if you want to submit it as a job to the HPC - so that it runs in the background without the need for your laptop to stay on - you can create a file (sHP063_errorcorrection) containing the following script:

In [ ]:
#PBS -lwalltime=32:00:00
#PBS -lselect=1:ncpus=20:mem=100gb

eval "$(~/anaconda3/bin/conda shell.bash hook)"

conda activate de_novo_assembly

cd /rds/general/user/gd1122/home/Diego_WGS/X204SC24082098-Z01-F002_01/01.RawData/sHP063
spades.py -1 sHP063_1.fq.gz -2 sHP063_2.fq.gz -o spades_error_corrected_reads -t 50 -m 500 --only-error-correction

conda deactivate

and you can submit it with:

In [ ]:
qsub sHP063_errorcorrection

And do the same for sHP067 in parallel. Full guide on the Imperial HPC and submit jobs guidance [here](https://icl-rcs-user-guide.readthedocs.io/en/latest/hpc/).

The next step is running the actual assembly on the error-corrected reads. Once more, this step can be computationally intensive so it is recommended to run it as a submitted job as above. For sHP063, I tried the following parameters for SPAdes (guide for command line options [here](https://ablab.github.io/spades/running.html)):

In [ ]:
spades.py -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
          -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
          -o spades_assembly_default --only-assembler

spades.py -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
          -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
          -o spades_assembly_careful --only-assembler --careful

spades.py -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
          -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
          -o spades_assembly_careful_kmers --only-assembler --careful -k 21,33,55,77

spades.py -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
          -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
          -o spades_assembly_isolate --only-assembler --isolate

spades.py -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
          -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
          -o spades_assembly_isolate_kmers --only-assembler --isolate -k 21,33,55,77

and for MEGAHIT (guide [here](https://github.com/voutcn/megahit/wiki/Assembly-Tips)):

In [ ]:
megahit -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
        -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
        -o megahit_assembly

megahit -1 spades_error_corrected_reads/corrected/sHP063_1.fq00.0_0.cor.fastq.gz \
        -2 spades_error_corrected_reads/corrected/sHP063_2.fq00.0_0.cor.fastq.gz \
        -o megahit_assembly_mincount --min-count 3

And did the same for sHP067.



We can now compare our results using two different programs, [QUAST](https://quast.sourceforge.net/docs/manual.html) and [BUSCO](https://busco.ezlab.org/busco_userguide.html). Quast is a tool to evaluate genome assemblies by computing various metrics and to compare genome assembly with a reference genome while BUSCO allows a measure for quantitative assessment of genome assembly based on evolutionarily informed expectations of gene content.

First we need to download a reference genome to use as comparison from NCBI. We can do this as follows:

In [ ]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz

Then, we can run QUAST:

In [ ]:
quast -o X204SC24082098-Z01-F002_01/01.RawData/sHP063/quastout_sHP063 -R GCF_000146045.2_R64_genomic.fna -G GCF_000146045.2_R64_genomic.gff -m 1000 \
-l "spades_default, spades_careful, spades_isolate, spades_careful_kmers, spades_isolate_kmers, megahit, megahit_mincount" \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_default/contigs.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_careful/contigs.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_isolate/contigs.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_careful_kmers/contigs.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_isolate_kmers/contigs.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/megahit_assembly/final.contigs.fa \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/megahit_assembly_mincount/final.contigs.fa 

From its report, we can see that overall the best assembly seems to be the one obtained with the --isolate flag, with or without the kmers options.
To further assess it, we can also run BUSCO:

In [ ]:
busco -i X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_isolate/contigs.fasta -m genome \
-o X204SC24082098-Z01-F002_01/01.RawData/sHP063/buscoout_sHP063 \
-l saccharomycetes_odb10 -c 16

	***** Results: *****

	C:99.1%[S:97.2%,D:1.9%],F:0.1%,M:0.7%,n:2137,E:0.1%	   
	2118	Complete BUSCOs (C)	(of which 2 contain internal stop codons)		   
	2077	Complete and single-copy BUSCOs (S)	   
	41	Complete and duplicated BUSCOs (D)	   
	3	Fragmented BUSCOs (F)			   
	16	Missing BUSCOs (M)			   
	2137	Total BUSCO groups searched		   

    Assembly Statistics:
	1471	Number of scaffolds
	1471	Number of contigs
	11795778	Total length
	0.000%	Percent gaps
	125 KB	Scaffold N50
	125 KB	Contigs N50
    
Then do the same with sHP067.

We now obtained contigs for both of parental species. We can try to improve the genome assembly by assembling and ordering contigs (contiguous sequences) into larger structures (scaffolds), hopefully representing chromosomes or entire genomes (NB: this is a hard task, especially only using short reads).
We use two programs to achieve this, [Ragout](https://github.com/mikolmogorov/Ragout/blob/master/docs/USAGE.md) and [RagTag](https://github.com/malonge/RagTag/wiki), and then we can compare their output using QUAST.

For Ragout, we have to create a "recipe file" that describes the run configuration and parameters:

In [ ]:
#reference and target genome names (required)
.references = scerR64
.target = sHP063

#paths to genome fasta files (required for Sibelia)
scerR64.fasta = path/to/file/GCF_000146045.2_R64_genomic.fna
sHP063.fasta = path/to/folder/X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_isolate/contigs.fasta

#reference to use for scaffold naming (optional)
.naming_ref = scerR64

and then run it with:

In [ ]:
# switch environment
conda deactivate
conda activate ragout_env

ragout X204SC24082098-Z01-F002_01/01.RawData/sHP063/recipe.txt \
-o X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragout_sHP063 --refine -t 16

We than try RagTag, with the scaffold module, first using the -C flag that concatenates unplaced contigs to make 'chr0' and then without it.

In [ ]:
# switch environment
conda deactivate
conda activate de_novo_assembly

ragtag.py scaffold GCF_000146045.2_R64_genomic.fna \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_careful/contigs.fasta \
-o X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragtag_scaffold_C_sHP063 -r -C

ragtag.py scaffold GCF_000146045.2_R64_genomic.fna \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/spades_assembly_careful/contigs.fasta \
-o X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragtag_scaffold_sHP063 -r

Then we run QUAST to compare the outputs:

In [ ]:
quast -o X204SC24082098-Z01-F002_01/01.RawData/sHP063/quastout_scaffold_sHP063 -R GCF_000146045.2_R64_genomic.fna -G GCF_000146045.2_R64_genomic.gff -m 1000 \
-l "ragout, ragtag_C, ragtag" \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragout_sHP063/sHP063_scaffolds.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragtag_scaffold_C_sHP063/ragtag.scaffold.fasta \
X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragtag_scaffold_sHP063/ragtag.scaffold.fasta

and do the same with sHP067.

It looks like in both cases the best reconstructed genome assembly is the one obtained with RagTag without the -C flag. These will now become our new reference genomes to use for the next step in the pipeline, variant calling. 

NB. Don't forget to deactivate the conda environment once you are done:

In [ ]:
conda deactivate

## Step 2: Variant calling

The next step  is to identify genomic variants in the evolved strains, including single nucleotide polymorphisms (SNPs), DNA insertions and deletions (indels), copy number variants (CNVs) and structural variants (SVs). 
Let's create a new folder for this and copy the relevant files that will be used in this step:

In [ ]:
mkdir variant_calling && cd variant_calling
mkdir sHP063 && mkdir sHP067

# copy new reference fasta for sHP063 and B2 evolved strain fastq reads
cp ../X204SC24082098-Z01-F002_01/01.RawData/sHP063/ragtag_scaffold_sHP063/ragtag.scaffold.fasta sHP063/
mv sHP063/ragtag.scaffold.fasta sHP063/sHP063_reference.fasta
cp ../X204SC24082098-Z01-F002_01/01.RawData/B2/B2_*.fq.gz sHP063/

# copy new reference fasta for sHP067 and R8 evolved strain fastq reads
cp ../X204SC24082098-Z01-F002_01/01.RawData/sHP067/ragtag_scaffold_sHP067/ragtag.scaffold.fasta sHP067/
mv sHP067/ragtag.scaffold.fasta sHP067/sHP067_reference.fasta
cp ../X204SC24082098-Z01-F002_01/01.RawData/R8/R8_*.fq.gz sHP067/

### 2.1 SNPs and indels

To identify SNPs and indels I am following [this tutorial](https://gencore.bio.nyu.edu/variant-calling-pipeline-gatk4/), which also provides an automated script to run all the steps using Nextflow on the HPC.

![variantcalling_GATK](variant_calling/Variant-Calling-Pipeline-GATK4.png)

Let's first set up the conda environment and nextflow, as described in the tutorial:

In [ ]:
git clone https://github.com/gencorefacility/variant-calling-pipeline-gatk4
    
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge

conda create -n variant_calling
conda activate variant_calling
conda install nextflow=22.10.0

We now need to set up the input parameters for the pipeline to run. Since we are using our own reference data, we first need to build index files for BWA (using BWA), a fasta index (using samtools), and a reference dictionary (using Picard Tools). These files will be located in the same directory as the reference fasta file.

In [ ]:
conda install -c bioconda bwa samtools picard

# we perform the steps for sHP063 first
cd sHP063/
bwa index sHP063_reference.fasta
samtools faidx sHP063_reference.fasta
picard CreateSequenceDictionary R=sHP063_reference.fasta O=sHP063_reference.dict

# and for sHP067
cd ../sHP067/
bwa index sHP067_reference.fasta
samtools faidx sHP067_reference.fasta
picard CreateSequenceDictionary R=sHP067_reference.fasta O=sHP067_reference.dict

cd .. # back to main directory '../variant_calling/'

We also need to download snpEff:

In [ ]:
wget -O snpEff_v4_3i_core.zip "https://sourceforge.net/projects/snpeff/files/snpEff_v4_3i_core.zip/download"
unzip snpEff_v4_3i_core.zip

Move the relevant scripts in our main directory:

In [ ]:
mv variant-calling-pipeline-gatk4/main.nf .
mv variant-calling-pipeline-gatk4/bin/parse_metrics.sh .

And modify the ```main.nf``` script to specify the correct file path for ```snpEff``` and ```parse_metrics.sh```:

In [ ]:
# add the full path to snpEff.jar within process snpEff (line 352)
java -jar /full/path/to/variant_calling/snpEff/snpEff.jar -v 

# add the full path to parse_metrics.sh within process qc (line 383)
/full/path/to/variant_calling/parse_metrics.sh ${pair_id} > ${pair_id}_report.csv

Finally we need to create the two ```.config``` files for B2 and R8 that specify all the parameters for the pipeline to run:

In [ ]:
// Required Parameters
params.reads = "/full/path/to/variant_calling/sHP063/*_{1,2}.fq.gz"
params.ref = "/full/path/to/variant_calling/sHP063/sHP063_reference.fasta"
params.outdir = "/full/path/to/variant_calling/sHP063/vc_out"
params.snpeff_db = "Saccharomyces_cerevisiae"
params.pl = "illumina"
params.pm = "novaseq"

// Set the Nextflow Working Directory
// By default this gets set to params.outdir + '/nextflow_work_dir'
workDir = params.outdir + '/nextflow_work_dir'

process {
    executor = 'pbs'

    clusterOptions = '-l walltime=32:00:00 -l select=1:ncpus=20:mem=100gb'
}

and create a similar file also for sHP067 / R8.

All is set. We can now call the script ```main.nf``` for each strain as follows:

In [ ]:
nextflow run main.nf -c nf_sHP063.config -with-singularity gencorefacility/variant-calling-pipeline-gatk4

nextflow run main.nf -c nf_sHP067.config -with-singularity gencorefacility/variant-calling-pipeline-gatk4

Nextflow will take care of submitting the jobs to the cluster for you and run all the scripts. The results will be saved in the output directory specificed in the config file.

On the Imperial HPC, it took around 1h / 1h30 for the whole script to run.

While the whole pipeline did run without errors, the snpEff step is not completely correct since the reference we are using (sHP063) does not use the same nomenclature as the reference snpEff uses. To overcome this, we can build our own database and run again snpEff using it.

We first need to create a .gff file for our de novo assembled genome. We can do it using [Liftoff](https://github.com/agshumate/Liftoff).

In [ ]:
conda install -c bioconda liftoff

liftoff -g ../GCF_000146045.2_R64_genomic.gff -o sHP063/sHP063_reference.gff -p 16 -copies \
sHP063/sHP063_reference.fasta ../GCF_000146045.2_R64_genomic.fna 

Now we build the snpEff database as follows:

In [ ]:
cd snpEff
mkdir data
mkdir data/sHP063

cp ../sHP063/sHP063_reference.gff data/sHP063_sc/genes.gff
cp ../sHP063/sHP063_reference.fasta data/sHP063_sc/sequences.fa

echo "sHP063_sc.genome : sHP063_sc" >> snpEff.config
java -jar snpEff.jar build -gff3 -v sHP063_sc

# double check the database has been added 
java -jar snpEff.jar databases | grep -i sHP063

and re-run the snpEff annotation step as follows:

In [ ]:
cd ../sHP063
mkdir snpEff_out

# SNPs
java -jar ../snpEff/snpEff.jar sHP063_sc vc_out/out/filtered_snps/B2_filtered_snps_2.vcf > snpEff_out/B2_filtered_snps.ann.vcf

# indels
java -jar ../snpEff/snpEff.jar sHP063_sc vc_out/out/filtered_indels/B2_filtered_indels_2.vcf > snpEff_out/B2_filtered_indels.ann.vcf

Finally repeat the same steps for sHP067 and R8.

In [ ]:
conda deactivate

### 2.2 CNVs

CNVs are a type of structural variation showing deletions or duplications in the genome. We will use [CNVpytor](https://github.com/abyzovlab/CNVpytor) to detect them.

Let's make new output folders to store the results of our analysis:

In [ ]:
mkdir sHP063/cnv && mkdir sHP063/cnv

Now we create a separate environment and install CNVpytor using pip:

In [ ]:
conda create -n cnvpytor_env python=3.12 -y
conda activate cnvpytor_env
pip install git+https://github.com/abyzovlab/CNVpytor.git

The first step is to configure our own reference genome, since CNVpytor installation includes only the human genomes hg19 and hg38. We can follow [these instructions](https://github.com/abyzovlab/CNVpytor/blob/master/examples/AddReferenceGenome.md):

In [ ]:
cd sHP063/cnv && cp ../sHP063_reference.fasta .
cnvpytor -root sHP063_file.pytor -gc sHP063_reference.fasta -make_gc_file

We don't have a mask file so we will skip that step. 

Now let's create the ```ref_genome_conf.py```:

In [ ]:
import_reference_genomes = {
    "sc_sHP063": {
        "name": "sHP063",
        "species": "Saccharomyces cerevisiae sHP063",
        "chromosomes": OrderedDict(
            [('NC_001133.9_RagTag', (229557, 'A')), ('NC_001134.8_RagTag', (807497, 'A')), ('NC_001135.5_RagTag', (317092, 'A')), 
             ('NC_001136.10_RagTag', (1526506, 'A')), ('NC_001137.3_RagTag', (570770, 'A')), ('NC_001138.5_RagTag', (265709, 'A')), 
             ('NC_001139.9_RagTag', (1086801, 'A')), ('NC_001140.6_RagTag', (553953, 'A')), ('NC_001141.2_RagTag', (421153, 'A')), 
             ('NC_001142.9_RagTag', (728491, 'A')), ('NC_001143.9_RagTag', (667136, 'A')), ('NC_001144.5_RagTag', (1057334, 'A')), 
             ('NC_001145.3_RagTag', (920445, 'A')), ('NC_001146.8_RagTag', (776296, 'A')), ('NC_001147.6_RagTag', (1080639, 'A')), 
             ('NC_001148.4_RagTag', (928892, 'A')), ('NC_001224.1_RagTag', (7178, 'M'))]),
        "gc_file":"/full/path/to/variant_calling/sHP063/cnv/sHP063_file.pytor",
    }
}

To identify the names and length of each chromosomes, you can use the following script in python (you will need the ```biopython``` package installed --> ```pip install biopython```):

In [ ]:
from Bio import SeqIO

def process_fasta(file_path):
    chromosomes = []
    for record in SeqIO.parse(file_path, "fasta"):
        chrom_name = record.id
        chrom_length = len(record.seq)
        tag = "A"
        
        chromosomes.append((chrom_name, (chrom_length, tag)))
    
    return chromosomes

fasta_file = "/path/to/variant_calling/sHP067/cnv/sHP063_reference.fasta"  # Replace with your FASTA file path
chromosome_data = process_fasta(fasta_file)
print(chromosome_data)

We can remove all the fragments (i.e. all chromosomes names not starting with NC_XXXXX_RagTag). Also we should modify the last chromosome type to mitochondrial - NC_001224.1_RagTag -> 'M'.

The same steps need to be repeated for sHP067.

Before starting the analysis, we also need to index and sort our reads:

In [ ]:
conda install samtools bwa -y

bwa index sHP063_reference.fasta 
bwa mem sHP063_reference.fasta ../B2_1.fq.gz ../B2_2.fq.gz > B2_alignment.sam # can take some time (up to 30min)
samtools view -Sb B2_alignment.sam > B2_alignment.bam
samtools sort B2_alignment.bam -o B2_sorted_alignment.bam
samtools index B2_sorted_alignment.bam 

Again, you will need to repeat this for sHP067 / R8.

We're now read to predict CNV regions. We can follow [this guide](https://github.com/abyzovlab/CNVpytor/wiki/2.-Read-Depth-Signal):

In [ ]:
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor \
-chrom NC_001133.9_RagTag NC_001134.8_RagTag NC_001135.5_RagTag NC_001136.10_RagTag NC_001137.3_RagTag NC_001138.5_RagTag NC_001139.9_RagTag NC_001140.6_RagTag NC_001141.2_RagTag NC_001142.9_RagTag NC_001143.9_RagTag NC_001144.5_RagTag NC_001145.3_RagTag NC_001146.8_RagTag NC_001147.6_RagTag NC_001148.4_RagTag NC_001224.1_RagTag \
-rd B2_sorted_alignment.bam 

# we can double-check we are using the right reference genome with
cnvpytor -root B2.pytor -ls

# calculate read depth histograms, GC correction and statistics type
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor -his 100 1000 10000 100000

# partitioning using mean-shift method
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor -partition 100 1000 10000 100000

# call CNV regions for each bin size
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor -call 100 > B2.calls.100.tsv
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor -call 1000 > B2.calls.1000.tsv
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor -call 10000 > B2.calls.10000.tsv
cnvpytor -conf sHP063_ref_genome_conf.py -root B2.pytor -call 100000 > B2.calls.100000.tsv

Then repeat the same steps to identify CNVs in R8 with respect to sHP067.